In [ ]:
import docx2txt
import os.path
import re
import pandas as pd
import win32com.client

In [ ]:
# define a message to be passed through later
def message(position):   # function that creates mesage to be sent for 
    msg = ("Dear Applicant,\n\n"       # date and time to be set manually
           " Thank you for applying for the position of " + str(position) + " with SomeCompany,"
           " we would like to interview you at <date and time> in our offices at 15 Grand Canal Plaza.\n"
           " If this is acceptable please email your acceptance, otherwise please let us know so we may"
           " schedule another time."
           " \n\nYours Sincerely,\n\n"
           "Human Resurces\n"
           "SomeCompany Dublin Branch"
           )
    return msg
# other material to be defined beforehand
df = pd.read_csv('C:\User\Desktop\ Job Descriptions.csv') # remember to specify path to CSV
patterns = pd.Series(data=df['Job Description'].tolist()) # convert job descriptions to list for matching
position = []

In [ ]:
#docs: http://stackoverflow.com/questions/2043980/using-win32com-and-or-active-directory-how-can-i-access-an-email-folder-by-name
outlook = win32com.client.Dispatch("Outlook.Application")
ons = outlook.GetNamespace("MAPI")
def findFolder(folderName,searchIn):
    
    lowerAccount = searchIn.Folders
    for x in lowerAccount:
        if x.Name == folderName:
            print 'found folder %s'%x.Name
            objective = x
            return objective
    
        
one = "john.doe@somecompany.com"
Folder1 = findFolder(one,ons)
Folder2 = findFolder('Inbox',Folder1)
Folder3 = findFolder('Another Example',Folder2)

messages = Folder3.Items
#for msg in messages:
    #print msg.Body

In [ ]:
# A series of functions:
# first to get position the person is applying for - this is done by cross referencing with Excel sheet containing the job title
def get_position(position, document, patterns):
    for pattern in patterns:
        if len(position) < 1: # so only one job description can be returned per person
            document.lower() # both are converted to lower case for comparison in case of errors
            pattern.lower()
            match = re.search(pattern, document)  # search for a job description (from CSV) in document
            if match == None: # to check if nothing matches the job descritption
                return None             # return none to prevent it from passing an empty into the group function - this breaks it     
            position = match.group(0)
            if position:         # if position has a value
                return position

# next check the both email adress and job title are present for email
def check_validity(email_address, position):
    if email_address == None and position != None:
        print 'No valid email for applicant of ', position
        return 1 # return 1 as it won't proceed in function if it is not = 0
    if position == None and email_address != None:
        print 'Unable to find valid position for applicant with email ', email_address
        return 1
    if position == None and email_address == None:
        return 1
    else:
        return 0

# function to send message
def send_message(email_address, position):
    obj = win32com.client.Dispatch("Outlook.Application") # make sure to redefine for creating a MailItem
    olMailItem = 0x0
    newMail = obj.CreateItem(olMailItem)
    newMail.Subject = "Your application to SomeCompany" # set Re: email subject title, could also refer to job description in email
    newMail.Body = message(position)   # function for body, to save space
    newMail.To = email_address         # parsed from document
    #attachment1 = "c:\\mypic.jpg"
    #newMail.Attachments.Add(attachment1)    # in case anyone wants to add attachments
    newMail.Send()
    print 'Position: ', position       # print to confirm what has been sent
    print 'SENT to: ', email_address            

# Note all these are done in one function so we don't have to define 'document' 
# outside the function; a subsequent function does not recognise the 'attribute' for attachment, 'lower' etc.
def auto_response(msg, position, patterns, count): 
    if msg.Attachments:      # so only if there are attachments               
        for att in msg.Attachments:
            if att.FileName.endswith('.docx'):   # As all attached C.V.s will be Word docs
                att.SaveAsFile(r"C:\User\Desktop" + '\\new.docx') # save file to desktop as new.docx
                rootdir = 'C:\User\desktop'
                document = docx2txt.process(os.path.join(rootdir,"new.docx"))
                # use Regular Expression to search for pattern around email address  
                match = re.search(r'[\w\.-]+@[\w\.-]+', document) # this gets email address from document (which is one string)
                if match == None:   # in case there is no email detected in the document
                    return None
                email_address = match.group() # assign to variable so it can be called upon 
                position = get_position(position, document, patterns)  # getting position
                valid = check_validity(email_address, position)       # checking if it is valid
                if valid == 0: # confirmation for notebook that email address has been correctly parsed and implemented
                    send_message(email_address, position)
                    count +=1 
                    return count # return count +1 everytime a succesful email is sent

In [ ]:
# This will go through each message in inbox and apply function
#- checking first if there is an attachment first
# emails should be cleared out from folder before this is run again - to prevent duplication
# as folder is only filled with emails from applicants through Mail Rules in Outlook
count = 0
for msg in messages:
    count_modified = auto_response(msg, position, patterns, count)
    
print "Emails sent Total: ", count_modified
# This will show any error messages below as well as confirming when messages have been sent, for what positions and to who